In [ ]:
import numpy as np
import os as os
import sys as sys
import multiprocessing as mp
import pandas as pd
import socket
import shutil as shutil

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

#sys.path.append("./package/")  # Since now we are in the Root Directory.
sys.path.insert(0,"./package/")  # Hack to get developer package
#from hmm_inference import HMM_Analyze   # Do not move. Should be after sys.path..

#from PackagesSupport.parallel_runs.helper_functions import prepare_path, create_folders, postprocess_iid
from hapsburg.PackagesSupport.hapsburg_run import hapsb_chrom
from hapsburg.PackagesSupport.parallel_runs.helper_functions import multi_run, split_up_roh_df  # Parallel Runs and forward ground truth
from hapsburg.PackagesSupport.pp_individual_roh_csvs import merge_called_blocks

midway2-0402.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


### Run the test case 
Vary Parameters and the suffixes to explore parameter range

In [42]:
### Prepare Parameter files and run
#### Create the parameters array for the starmap:
iids = ["iid" + str(i) for i in range(0,100)]   # List of iids to iterate over
### Create list of IIDs and of Folders

ch = 3 # For test case here: Only do Chromosome #3
n_ref = 2504  # 2504 All 503 Europe/TSI 504 EAS
diploid_ref = True
save = True
save_fp = False

#exclude_pops = ["TSI", ]
exclude_pops = []
e_model = "readcount"
p_model = "MosaicHDF5"  
readcounts = True
destroy_phase = False
post_model = "Standard"
h5_path1000g = "./Data/1000Genomes/HDF5/1240kHDF5/all1240gzip/chr" # Switch: Eur1240chr for classic
meta_path_ref = "./Data/1000Genomes/Individuals/meta_df_all.csv"  # meta_df.csv for full 1000G
base_path="./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/" # Simulated Mosaics
#base_path="./Simulated/1000G_Mosaic/CLM/" # Simulated Mosaics

roh_in = 1   #1  New: 0.1
roh_out = 20 # 20 Good: 10
roh_jump = 300
e_rate = 0.01  # The Error Rate
e_rate_ref = 0.0
max_gap = 0.0 # Gap Merging. In M
cutoffs = [0.99]
#cutoffs = [0.9]
l_cutoff = 0.02
logfile = True
#lengths = [0, 2, 4, 6, 8, 10] # Which Block Lengths to test
lengths = [0,4]  # 0, 4 Relevant ones for key performance testing
#prefix_out = "eas_ref/" # Check as well below in loop!!!

folders = [base_path + "ch" + str(ch) + "_" + str(int(l)) + "cm/" for l in lengths]  # Prepare Length folders. # "cm/missing/5/"

#########################################################
### Create the List of Parameter Lists (input for starmap)
prms = []
for f in folders: 
    path_targets = f + "data.h5"
    base_out_folder = os.path.join(f, "output", "")
    
    for cutoff in cutoffs:
        prefix_out = "c" + str(cutoff).replace("0.", "") + "/"
        #prefix_out = "c9/"  # "eas_ref.jump500/"
    
        for iid in iids:
            new_par = [iid, ch, save, save_fp, n_ref, diploid_ref, exclude_pops, e_model, p_model, readcounts, destroy_phase,
            post_model, path_targets, h5_path1000g, meta_path_ref, base_out_folder, prefix_out,
            roh_in, roh_out, roh_jump, e_rate, e_rate_ref, max_gap, cutoff, l_cutoff, logfile]
            prms.append(new_par)  # Append to the Parameters
        
assert(len(prms[0])==26)   # Sanity Check
print(len(prms))

200


In [43]:
prefix_out

'c99/'

### Test Run of single indivdual
Takes about 45 seconds for one run

In [ ]:
%%time
assert(logfile==False)
multi_run(hapsb_chrom, [prms[119]], processes = 1)

### Run full list of parameters
Watch out: 20 or even 10 does not work.

With runtime updates: Shoots up to 10 gb memory per process

In [31]:
%%time
assert(logfile==True) # Sanity Check
multi_run(hapsb_chrom, prms[:], processes = 10) # #For all ref: 4 for everything, 8 for 0.5x. For Europe only multiply x5

Running 400 total jobs; 10 in parallel.
Starting Pool of multiple workers...
Set Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid40/chr3/c95/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid0/chr3/c95/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid20/chr3/c95/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid50/chr3/c95/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid30/chr3/c95/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid60/chr3/c95/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid80/chr3/c95/hmm_run_log.txt

Set Output Log path: ./Simulated/1000G_Mosaic/TSI6/lambda_rc1.0/ch3_0cm/output/iid90/chr3/c95/hmm_run_log.txt

Set Output Log path: ./Simulated/1000G_Mosaic/TS

## Split up ground truth ROH into according folders
Modify function if multiple prefix_out are given

Takes about 30s for 600 prms  
Takes about 4s for 200 prms! (including 100 0s)

In [26]:
def copy_gt():
    """Split up ground truth roh.csv into
    ground truth for each folder"""
    for f in folders:
        for iid in iids[:]:
            for cutoff in cutoffs:
                prefix_out = "c" + str(cutoff).replace("0.", "") + "/" # Comment out
                #prefix_out="test/"
                path_out = os.path.join(f, "output", iid, "chr"+str(ch), prefix_out)
                split_up_roh_df(f, path_out, iid, 
                                file_in="roh_info.csv", 
                                file_out="roh_gt.csv")

In [32]:
%%time
copy_gt()

CPU times: user 2.08 s, sys: 150 ms, total: 2.23 s
Wall time: 4.2 s


# Run various degrees of downsampling for read-count data

In [30]:
### Prepare Parameter files and run
#### Create the parameters array for the starmap:
iids = ["iid" + str(i) for i in range(0,100)]   # List of iids to iterate over
ch = 3 # For test case here: Only do Chromosome #3
n_ref = 2504  # 2504 All 503 Europe/TSI
save = True
save_fp = False
diploid_ref = True

exclude_pops = ["TSI", ]
 
e_model = "haploid"  # "readcount"
p_model = "MosaicHDF5"  
readcounts = False
destroy_phase = True

post_model = "Standard"
h5_path1000g = "./Data/1000Genomes/HDF5/1240kHDF5/all1240int8/chr" # Switch: Eur1240chr
meta_path_ref = "./Data/1000Genomes/Individuals/meta_df_all.csv"  # meta_df.csv for full 1000G

prefix_out = "ph_c999/"
roh_in = 1   #1  New: 0.1
roh_out = 20 # 20 Good: 10
roh_jump = 300
e_rate = 0.01  # The Error Rate
e_rate_ref = 0.0
max_gap = 0.01 # Gap Merging. In M
#cutoffs = [0.9, 0.99, 0.9999, 0.99999]
#cutoffs = [0.996, 0.997, 0.998, 0.999]
#cutoffs = [0.998, 0.999, 0.9995]
cutoff = 0.999
l_cutoff = 0.01

logfile = True
rcs = ["0.1", "0.2", "0.3", "0.4", "0.5", "0.6"] # Various degrees of missing data
rcs = ["0.3"]
lengths = [0, 4]  # Relevant ones for key performance testing
base_path = "./Simulated/1000G_Mosaic/TSI5/lambda_rc" #0.4/ch3_4cm/data.h5"

#########################################################
### Create the List of Parameter Lists (input for starmap)

prms = []
for rc in rcs:
    folders = [base_path + rc + "/ch" + str(ch) + "_" 
               + str(int(l)) + "cm/" for l in lengths]
    
    for f in folders: 
        path_targets = f + "data.h5"
        base_out_folder = os.path.join(f, "output", "")

        for iid in iids:
            new_par = [iid, ch, save, save_fp, n_ref, diploid_ref, exclude_pops, e_model, p_model, readcounts, destroy_phase,
            post_model, path_targets, h5_path1000g, meta_path_ref, base_out_folder, prefix_out,
            roh_in, roh_out, roh_jump, e_rate, e_rate_ref, max_gap, cutoff, l_cutoff, logfile]
            prms.append(new_par)  # Append to the Parameters
        
assert(len(prms[0]) == 26)   # Sanity Check
print(len(prms))

200


### Test run of single individual
Check how long a single individual runs

In [ ]:
prms[1400]

In [ ]:
%%time
assert(logfile==False)
multi_run(hapsb_chrom, [prms[1400]], processes = 1)

# Run full set of replicates
Single replicate: ~30s. 
Depending on how much downsampling was done.

100 Replicates with 10 cores ~ 2-5 min

In [31]:
%%time
assert(logfile == True) # Sanity Check
multi_run(hapsb_chrom, prms[:], 
          processes = 12) 

Running 200 total jobs; 12 in parallel.
Starting Pool of multiple workers...
Set Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid15/chr3/ph_c999/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid10/chr3/ph_c999/hmm_run_log.txt

Set Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid0/chr3/ph_c999/hmm_run_log.txt
Set Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid25/chr3/ph_c999/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid20/chr3/ph_c999/hmm_run_log.txt

Set Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid5/chr3/ph_c999/hmm_run_log.txt
Set Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid40/chr3/ph_c999/hmm_run_log.txtSet Output Log path: ./Simulated/1000G_Mosaic/TSI5/lambda_rc0.3/ch3_0cm/output/iid30/chr3/ph_c999/hmm_run_log.txtSet Output Log 

In [25]:
def copy_gt():
    """Split up ground truth roh.csv into
    ground truth for each folder"""
    for rc in rcs:
        folders = [base_path + rc + "/ch" + str(ch) + "_" 
                   + str(int(l)) + "cm/" for l in lengths]
        for f in folders:
            for iid in iids[:]:
                #prefix_out = "c" + str(cutoff).replace("0.", "") + "/" # Comment out
                prefix_out="ph_c999/"
                path_out = os.path.join(f, "output", iid, "chr" + str(ch), prefix_out)
                split_up_roh_df(f, path_out, iid, 
                                file_in="roh_info.csv", 
                                file_out="roh_gt.csv")

In [26]:
%%time
copy_gt()

CPU times: user 1.07 s, sys: 73 ms, total: 1.14 s
Wall time: 3.79 s


In [28]:
print("Hello? Blizzard?")

Hello? Blizzard?
